


## This notebook identifies mutations in the Coronavirus Delta and Omicron Spike Protein from the original Wild-Type (Wuhan)  which is used as reference. 

### Uses NCBI GenBank Accession Number for Spike and uses Entrez.efetch to obtain sequences from GenBank. Uses Clustal for alignment of sequences. 

Wild-type: YP_009724390 (used as reference)

Delta : QWK65230

Omicron: UFO69279


### References / notes:
1. This notebook identifies mutations in the Delta and Omicron Spike Protein. Mutations are listed in following (specifically, Figure 1): https://onlinelibrary.wiley.com/doi/full/10.1002/jmv.27526 . 
2. Additional, the following includes figure identifying specific mutations https://asm.org/Articles/2021/December/How-Ominous-is-the-Omicron-Variant-B-1-1-529 . 
3. Note differences in the above in interpretation; e.g. on Omicron around position 211. 

### Inputs and Outputs:
Inputs (retrieved from Genbank using the NCBI accession number in fasta format):
* Reference sequence (in this example, it is the Coronavirus Wuhan Spike Protein, aka wild-tpye)
* Subject sequence (in this case, the Delta Spike Protein)

Outputs:
* List of mutations in mutation format; e.g. T478K:   T in position 478 in reference was mutated to K in subject. The final mutations are mapped back to the original reference sequence; i.e. the 478 is the position in the referenced sequence (1-based).

### Process Steps

1. Execute setup - Imports, etc.
2. Get input sequences from Genbank
3. Align input sequences (uses Clustal)
4. Find mutations in the aligned sequences
5. Map mutations back to the reference sequence

### Expected Results (Wikipedia sources):

Omicron Spike protein mutations: A67V, Δ69-70, T95I, G142D, Δ143-145, Δ211, L212I, ins214EPE, G339D, S371L, S373P, S375F, K417N, N440K, G446S, S477N, T478K, E484A, Q493R, G496S, Q498R, N501Y, Y505H, T547K, D614G, H655Y, N679K, P681H, N764K, D796Y, N856K, Q954H, N969K, L981F

Delta spike protein mutations: T19R, G142D, Δ156-157, R158G, L452R, T478K, D614G, P681R, D950N

### Calculated results:

Omicron: Have differences around Δ211, L212I, ins214EPE (the 'insertion hotspot')

Delta: calculated exactly.


## Step 1. Imports and Setup

In [1]:
import Bio                        
from Bio import  Entrez                                         # needed for retrieval from Genbank
from Bio.Align.Applications import ClustalOmegaCommandline      # needed for alignment
import tempfile


In [2]:
# Define class for mutation

class Mutation:   
    genbank_reference = ("name", "number")        # NCBI GenBank Accession Number of subject sequence (e.g. Delta Spike)
    genbank_subject = ("name", "number" )         # NCBI GenBank Accession Number of reference sequence (e.g. Wuhan Spike)
    
    def __init__(self, index, typex, align_loc, ref_value, subj_value):

        self.index = index             # mutation number (sequential assigned: 1,2,3 )
        self.typex = typex             # type = ins, del, sub  (insert, deletion, substution)
        self.ref_value = ref_value     # value in the ref sequence that is mutated (could be more than one)
        self.subj_value = subj_value   # value in subject that is changed from base
        self.align_loc = align_loc     # location in aligned sequence (1 base)
        self.ref_loc = 0               # location in reference sequence (1 base)
  
    def __str__(self):
        return f"Index:  {self.index} ,{self.align_loc}, {self.ref_loc} , {self.typex}, {self.ref_value}, '-->' {self.subj_value}"



In [3]:
# Parameters  (comment out all but one subject for each execution)

#genbank_subject = ("Omicron", "UFO69279")        # this is the subject.
#genbank_subject = ("Delta", "QWK65230")       # this is the subject.
#genbank_subject = ("Gamma", "QHD43416")       # this is the subject.
#genbank_subject = ("Beta", "UJB55404")       # this is the subject.

# Select subject and Reference
genbank_subject = ("Omicron", "UFO69279")        # this is the subject.
genbank_reference = ("Wild-type", "YP_009724390")  # this is the reference sequence (Wuhan origins).

##  Step 2. Get and save input sequences (Reference and Subject)


In [4]:
def read_seq(idX,database):     # Read sequence from Genbank
    """
    Inputs:   (a) NCBI accession number (b) database ("protein", etc.)
    Process:  (a) Retrieve seqence (header and sequence)  ---- Using Entrez (see references / link for more info)
    Outputs:  (a) sequence header (b) sequence
    """
    hdrX, seqX = '' , ''                          # output header and sequence
    fetch_handle = Entrez.efetch(database, id=idX, rettype='fasta', retmode="text")    # input
                                                           
    for record in fetch_handle:        # retrieved from database
        if len(record) == 0: break     # end
        if record[0] ==">":            #
            hdrX = record
        else: 
            seqX = seqX + record     # sequence record
    print(f"\nHdr: {hdrX} *** Sequence length: {len(seqX)}")       
    return hdrX,  seqX                      # header,  sequence 


In [5]:
# retrieve and write input sequences to a single fasta file for input to Clustal. 
# This will be a temporary file used to pass data. 

print ("Header and length of sequence for each input (possible NCBI warning due to lack of specified email address):")
# 1. Get Reference (first input)
ref_hdr, ref_seq = read_seq(genbank_reference[1],'protein')  # Retrieve sequence data from database (accession number and database)

# 2. Get Subject (second input)
sub_hdr, sub_seq = read_seq(genbank_subject[1],'protein')    # Retrieve sequence data from database (accession number and database)

temp_seqs = ref_hdr + ref_seq + sub_hdr + sub_seq            # format for Clustal    

# 3. Write output for Clustal using temporary file
tmp = tempfile.NamedTemporaryFile()       # create and write sequences to a single temporary file 
with open(tmp.name, 'w') as f:
    f.write(temp_seqs.strip()) 



Header and length of sequence for each input (possible NCBI warning due to lack of specified email address):


/home/raymond/anaconda3/lib/python3.9/site-packages/Bio/Entrez/__init__.py:658: UserWarning: 
Email address is not specified.

To make use of NCBI's E-utilities, NCBI requires you to specify your
email address with each request.  As an example, if your email address
is A.N.Other@example.com, you can specify it as follows:
   from Bio import Entrez
   Entrez.email = 'A.N.Other@example.com'
In case of excessive usage of the E-utilities, NCBI will attempt to contact
a user at the email address provided before blocking access to the
E-utilities.
  warnings.warn(



Hdr: >YP_009724390.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]
 *** Sequence length: 1293

Hdr: >UFO69279.1 surface glycoprotein [Severe acute respiratory syndrome coronavirus 2]
 *** Sequence length: 1290


## Step 3.  Align sequences for both reference and subject (Uses Clustal)

In [6]:
# Use clustal file that has aligned subject to reference  (2 sequences in fasta format)
# The program, 'clustal-align' was used
# Assumes input sequence 1 is the base (reference) and sequence 2 is the subject
# Output is subject align sequence, s_align and reference align sequence, r_align

# input is a single fasta file (tmp.name) with two sequences

cline = ClustalOmegaCommandline( cmd='clustalo',  infile = tmp.name,  auto = True , seqtype = 'protein')

aligned_seqs, stderr = cline()        

hold = aligned_seqs.split('>')[1]           # get first sequence which is reference (ignore header)
r_align = ''.join(hold.split('\n')[1:-1])   # get rid of line feeds

hold = aligned_seqs.split('>')[2]           # get second sequence which is subject
s_align = ''.join(hold.split('\n')[1:-1])   # get rid of line feeds

print('Length of aligned sequences should be equal: ',len(r_align), len(s_align) )


Length of aligned sequences should be equal:  1275 1275


## Step 4. Find the mutations in aligned sequences (Reference to Subject)

In [7]:
# Find mutation in aligned sequences
def mute_find(posM, query,ref):
    """
    Inputs
        pos   = position of mutation detection
        query = query seq (subject)
        ref   = reference (subject and ref are entire aligned potein sequences of equal lengths)
    
    Outputs:
        typeM = type of mutation (ins, del, sub)
        fromM = value of mutation in reference sequence (amino acid)
        toM   = value of mutation in query sequence (amino acid)
    """
    typeM,fromM, toM = '','',''   # initialize output parameters 
    mutM= ''                      # hold field to build 

    # Address deletion

    if ref[posM] != '-' and query[posM] == '-'   :   # if so, deletion
        fromM = ref[posM]                               # will be 'x'
        toM = query[posM]                               # will be '-'
        typeM = 'del'
        return typeM, fromM, toM
    
    # Address insertion
 
    if ref[posM] == '-' and query[posM] != '-'   :   # if so, insertion
        fromM = ref[posM]                               # will be '-'
        toM = query[posM]                               # will be 'x'
        typeM = 'ins'
        return typeM, fromM, toM

    # Address substution 

    if ref[posM] != '-'  and  query[posM] != ref[posM] :   # if so, then simple substitution
        fromM = ref[posM]
        toM = query[posM]
        typeM = 'sub'
        return typeM, fromM, toM
 
    print('Should not be here 1', posM, ref[posM:posM+2], "  ", query[posM+1] )           
    return typeM, fromM, toM


In [8]:
# Develop mutations
r_align = r_align + '999'    # add '999' as end to solve edge problem 
match = 0                    # counter for number of matches
pos = 0                      # position within aligned sequences
mutation_list = []           # list of mutations (instances of Mutation class)

while pos < len(s_align):          # increment through aligned sequences looking for mutations
    
    if s_align[pos] == r_align[pos]:         # does it match (i.e. no mutation)
        match += 1                           # increment number of matches
        pos += 1                             # increment position
        
    else:                                    # not a match, identify mutation

        typeX, fromX, toX = mute_find(pos, s_align, r_align)    # there's a mutation in this area 'i'; identify it
        
        #1 Handle substitutions
        if typeX == 'sub':               # a substitution
            if fromX != toX:    # don't substitute like amino acids (e.g. don't N -> N)
                mutation_list.append(Mutation(len(mutation_list)+1,  typeX, pos+1, fromX, toX))  # add mutation
            else:   # don't substitute like amino acids (e.g. don't N -> N) --- don't create mutation
                typeX=''           # --- don't create mutation (but advance pos by 1 by blanking type)   
        
        #2 Handle inserts and deletions 
        else:
            mutation_list.append(Mutation(len(mutation_list)+1,  typeX, pos+1, fromX, toX))  # add mutation
       
        #3 increment position        
        pos = pos + 1         
print('Number of mutations:', len(mutation_list), '  *Consecutive inserts and deletes are not consolidated')
print('Number of matching elements:', match)

Number of mutations: 40   *Consecutive inserts and deletes are not consolidated
Number of matching elements: 1235


In [9]:
# when there are multiple sequencial deletions or insertions, consolidate into one mutation
prior_type = 'xxx'
idx = 0
while idx <=   len(mutation_list)-1:
    # print(prior_type, mutation_list[idx].index, mutation_list[idx].typex, mutation_list[idx].align_loc, mutation_list[idx].ref_value, mutation_list[idx].subj_value)
    if mutation_list[idx].typex == 'sub':
        prior_type = 'sub'    # set prior, move to next
        idx += 1        
    else:   
        if mutation_list[idx].typex != prior_type:   # not equal then not repeat
            prior_type = mutation_list[idx].typex    # set prior, move to next
            idx += 1
        else:                          # equal; therefore, repeat. Consolidate    
            mutation_list[idx-1].ref_value = mutation_list[idx-1].ref_value + mutation_list[idx].ref_value
            mutation_list[idx-1].subj_value = mutation_list[idx-1].subj_value + mutation_list[idx].subj_value
            mutation_list = mutation_list[:idx] + mutation_list[idx+1:]  # remove entry
            prior_type = mutation_list[idx-1].typex    # set prior, move to next
            # don't increment as one was deleted
    
for i,mute in enumerate(mutation_list):       # renumber mutations 
    mute.index = i+1
print('Number of mutations:', len(mutation_list), '  *Consecutive inserts and deletes are consolidated')

Number of mutations: 36   *Consecutive inserts and deletes are consolidated


## Step 5.  Map mutations to reference sequence.

In [10]:
# The above list of mutations (muts_in_align) are locations in the aligned sequence. 
# We must have corresponding locations in the referenced sequence (Base Wuhan).

adjust = 10                           # length of search field (somewhat arbitary)
ref_seq = ref_seq.replace('\n', '')   # this is original (non-aligned) reference sequence   

print (f"{genbank_subject[0]} Mutations (type, loc, change from ref),   * note, these positions are base 1 in reference")
for count, mute in enumerate(mutation_list):
    
    # choose a search string (to avoid edge problem) and also can't have '-' that appear as in base aligned as subject inserts
              # mute.align_loc is location of mutation in aligned base
    r1 = mute.align_loc 
    r2 = mute.align_loc + adjust                    # go right with search string
    
    if r2 > len(ref_seq):                           # off the right edge
        r1 = mute.align_loc - adjust                # if yes, go left with search string
        r2 = mute.align_loc
            
    search_str = r_align[r1:r2-1]
    search_str = search_str.replace('-', '')        # del the mutation insert character from search string
    
    loc = ref_seq.find(search_str)                  # search 
    if loc == -1:                    # no find string?
        print(count+1,mute.typex, mute.align_loc, mute.ref_loc, mute.ref_value, mute.subj_value, "NO GO", search_str)
        print('***  ',search_str)
    else:                            # found string; adjust location based on direction of search and save loc in mute
        if r1 < mute.align_loc :  mute.ref_loc = loc + adjust
        else:                     mute.ref_loc = loc
    print(count+1,mute.typex, mute.ref_loc, mute.ref_value, '-->', mute.subj_value)
    

Omicron Mutations (type, loc, change from ref),   * note, these positions are base 1 in reference
1 sub 67 A --> V
2 del 69 HV --> --
3 sub 95 T --> I
4 sub 142 G --> D
5 del 143 VYY --> ---
6 sub 211 N --> I
7 sub 212 L --> V
8 ins 212 -- --> RE
9 sub 213 V --> P
10 sub 214 R --> E
11 sub 339 G --> D
12 sub 371 S --> L
13 sub 373 S --> P
14 sub 375 S --> F
15 sub 417 K --> N
16 sub 440 N --> K
17 sub 446 G --> S
18 sub 477 S --> N
19 sub 478 T --> K
20 sub 484 E --> A
21 sub 493 Q --> R
22 sub 496 G --> S
23 sub 498 Q --> R
24 sub 501 N --> Y
25 sub 505 Y --> H
26 sub 547 T --> K
27 sub 614 D --> G
28 sub 655 H --> Y
29 sub 679 N --> K
30 sub 681 P --> H
31 sub 764 N --> K
32 sub 796 D --> Y
33 sub 856 N --> K
34 sub 954 Q --> H
35 sub 969 N --> K
36 sub 981 L --> F
